# Trading Agent Analysis Notebook

This notebook demonstrates the reinforcement learning-based trading agent's capabilities, including:
- Model configuration and training
- Performance visualization
- Technical analysis
- Portfolio metrics

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ta
from datetime import datetime, timedelta
from typing import Dict, Any, List, Optional, Union, Tuple
from core.base_agent import UnifiedTradingAgent
from core.visualization import TradingVisualizer
import optuna
import logging

# Configure logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

## Initialize Trading Agent

Set up the trading agent with initial configuration.

In [ ]:
# Initialize the trading agent
model = UnifiedTradingAgent()

# Set environment parameters
env_params = {
    'initial_balance': 10000,
    'transaction_cost': 0.01,
    'use_position_profit': False,
    'use_holding_bonus': False,
    'use_trading_penalty': False
}

# Set PPO parameters
ppo_params = {
    'learning_rate': 3e-4,
    'n_steps': 512,
    'batch_size': 128,
    'n_epochs': 5,
    'gamma': 0.99,
    'clip_range': 0.2,
    'target_kl': 0.05
}

## Training Configuration

Set up the training period and stock symbol.

In [ ]:
# Configure training parameters
stock_name = "AAPL"  # Change this to your desired stock
train_start_date = datetime.now() - timedelta(days=365 * 5)
train_end_date = datetime.now() - timedelta(days=365 + 1)

# Train the model
metrics = model.train(
    stock_name=stock_name,
    start_date=train_start_date,
    end_date=train_end_date,
    env_params=env_params,
    ppo_params=ppo_params
)

# Display training metrics
print("Training Metrics:")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
print(f"Maximum Drawdown: {metrics['max_drawdown']:.2%}")
print(f"Sortino Ratio: {metrics['sortino_ratio']:.2f}")
print(f"Volatility: {metrics['volatility']:.2%}")
print(f"Total Return: {metrics['total_return']:.2%}")
print(f"Final Portfolio Value: ${metrics['final_value']:,.2f}")

## Model Testing and Visualization

Test the trained model and visualize its performance.

In [ ]:
# Configure test period
test_start_date = datetime.now() - timedelta(days=365)
test_end_date = datetime.now()

# Test the model
test_results = model.test(
    stock_name=stock_name,
    start_date=test_start_date,
    end_date=test_end_date,
    env_params=env_params,
    ppo_params=ppo_params
)

# Display test metrics
metrics = test_results['metrics']
print("\nTest Results:")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
print(f"Maximum Drawdown: {metrics['max_drawdown']:.2%}")
print(f"Sortino Ratio: {metrics['sortino_ratio']:.2f}")
print(f"Information Ratio: {metrics['information_ratio']:.2f}")
print(f"Volatility: {metrics['volatility']:.2%}")
print(f"Final Portfolio Value: ${test_results['portfolio_history'][-1]:,.2f}")

## Technical Analysis Visualization

In [ ]:
# Initialize visualizer
visualizer = TradingVisualizer()
visualizer.show_rsi = True
visualizer.show_sma20 = True
visualizer.show_sma50 = True
visualizer.rsi_period = 14

# Fetch and prepare data
portfolio_data = model.data_handler.fetch_data(stock_name, test_start_date, test_end_date)
portfolio_data = model.data_handler.prepare_data()

if stock_name in portfolio_data:
    data = portfolio_data[stock_name]
    
    # Create and display technical analysis chart
    main_chart = visualizer.create_single_chart(stock_name, data)
    main_chart.show()
    
    # Create and display cumulative returns
    cum_returns_fig = visualizer.plot_cumulative_returns({stock_name: data})
    cum_returns_fig.show()
    
    # Create and display drawdown
    drawdown_fig = visualizer.plot_drawdown({stock_name: data}, stock_name)
    drawdown_fig.show()
    
    # Create and display performance and drawdown combined
    perf_dd_fig = visualizer.plot_performance_and_drawdown({stock_name: data}, stock_name)
    perf_dd_fig.show()

## Agent Actions Analysis

In [ ]:
# Display agent actions plot
test_results['action_plot'].show()

# Display combined price and actions plot
test_results['combined_plot'].show()

# Plot returns distribution
fig = go.Figure(data=[go.Histogram(x=test_results['returns'], nbinsx=50)])
fig.update_layout(
    title="Returns Distribution",
    xaxis_title="Return",
    yaxis_title="Frequency",
    showlegend=True
)
fig.show()

# Plot drawdown over time
values = np.array(test_results['portfolio_history'])
peak = np.maximum.accumulate(values)
drawdowns = (peak - values) / peak
fig = go.Figure(data=[go.Scatter(y=drawdowns, mode='lines', name='Drawdown')])
fig.update_layout(title="Drawdown Over Time")
fig.show()